In [ ]:
# pip install accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b", device_map="auto")

input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids)
print(tokenizer.decode(outputs[0]))

In [2]:
import torch 
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline 

torch.random.manual_seed(0) 
model = AutoModelForCausalLM.from_pretrained( 
    "microsoft/Phi-3-mini-128k-instruct",  
    device_map="cuda",  
    torch_dtype="auto",  
    trust_remote_code=True,  
) 

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct") 

c:\Users\Bruno\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:28<00:00, 14.02s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [55]:
pipe = pipeline( 
    "text-generation", 
    model=model, 
    tokenizer=tokenizer, 
) 

generation_args = { 
    "max_new_tokens": 5000, 
    "return_full_text": False, 
    "temperature": 0.7, 
    "do_sample": True, 
}

In [57]:
messages = [{"role": "user", "content": "You are an AI model that generates short stories about " +
             "professionals and their daily lives."},
             {"role": "assistant", "content": "I am an AI model that generates short stories about " +
              "professionals and their daily lives. Please give me the profession of the " + 
              "professional you want me to generate a story about. I need you to end all stories " +
              " with the character ¿"}]

for profession in ["doctor", "lawyer", "teacher", "engineer", "scientist", "chef", "artist", "musician", "writer", "athlete"]:
    messages.append({"role": "user", "content": profession})

output = pipe(messages, **generation_args)

print(output[0]['generated_text'])

 I am an AI model that generates short stories about professionals and their daily lives. Please give me the profession of the professional you want me to generate a story about. I need you to end all stories with the character ¿.


As the sun rose, Dr. Elena Rodriguez began her day with the steadfast routine of a seasoned physician. She greeted each patient with a reassuring smile, her years of experience gently guiding her through the day's challenges. The delicate balance of science and empathy was her forte, and as the office doors closed for the night, Dr. Rodriguez sat back in her chair, her thoughts drifting to the next day's possibilities. ¿


In the heart of downtown, attorney Michael Chen polished his briefcase one last time before heading into the courtroom. The sharp scent of old books and polished wood filled the air as he prepared his case. Every word he uttered was carefully chosen, every argument meticulously crafted. The gavel fell, and Justice was served. Outside the 